# Load data

In [7]:
import json
from collections import Counter
from pprint import pprint

# Đọc file JSON
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

dataset = load_json('dataset_multiwoz.json')

# Get raw dataset

In [ ]:
data_train = []
data_validation = []
data_test = []

for split in ["train", "validation", "test"]:
    if split == "train":
        data = data_train
    elif split == "validation":
        data = data_validation
    elif split == "test":
        data = data_test

    for index_dialogue in range(len(dataset[split]["dialogue_id"])):
        dialogue_id = dataset[split]["dialogue_id"][index_dialogue]
        for index_turn in range(len(dataset[split]["turns"][index_dialogue]['turn_id'])):
            record = [None] * 4
            record[0] = dialogue_id
            record[1] = dataset[split]["turns"][index_dialogue]["turn_id"][index_turn]
            record[2] = dataset[split]["turns"][index_dialogue]["utterance"][index_turn]
            record[3] = dataset[split]["turns"][index_dialogue]["dialogue_acts"][index_turn]["span_info"]
            data.append(record)


In [9]:
import re

def create_bio_labels(utterance, spans, slot_names):
    # Tách câu thành các token
    tokens = re.findall(r'\w+|[^\w\s]', utterance, re.UNICODE)
    
    # Khởi tạo nhãn BIO cho các token
    bio_labels = ['O'] * len(tokens)
    
    # Duyệt qua từng span
    for start, end, slot_name in zip(spans['span_start'], spans['span_end'], slot_names):
        # Xác định vị trí token đầu tiên và cuối cùng của span
        token_start = len(re.findall(r'\w+|[^\w\s]', utterance[:start]))
        token_end = len(re.findall(r'\w+|[^\w\s]', utterance[:end]))
        
        # Gán nhãn BIO cho các token trong span
        if token_start < len(tokens):
            bio_labels[token_start] = f'B-{slot_name}'
        for i in range(token_start + 1, min(token_end, len(tokens))):
            bio_labels[i] = f'I-{slot_name}'
    
    return tokens, bio_labels

import numpy as np

def get_raw_data(data):
    # data is data_train, data_validation, or data_test
    X_token_classification = []
    y_token_classification = []
    y_act = []

    # Tạo từ điển ánh xạ nhãn BIO thành chỉ số
    bio_labels = [
        "O",
        "B-address", "B-area", "B-arriveby", "B-bookday", "B-bookpeople", "B-bookstay", "B-booktime",
        "B-choice", "B-day", "B-department", "B-departure", "B-destination", "B-duration", "B-entrancefee", 
        "B-food", "B-leaveat", "B-name", "B-openhours", "B-phone", "B-postcode", "B-price", "B-pricerange", 
        "B-ref", "B-stars", "B-trainid", "B-type", "I-address", "I-area", "I-arriveby", "I-bookday", 
        "I-bookpeople", "I-bookstay", "I-booktime", "I-choice", "I-day", "I-department", "I-departure", 
        "I-destination", "I-duration", "I-entrancefee", "I-food", "I-leaveat", "I-name", "I-openhours", 
        "I-phone", "I-postcode", "I-price", "I-pricerange", "I-ref", "I-stars", "I-trainid", "I-type"
    ]

    # Tạo từ điển ánh xạ nhãn BIO thành chỉ số
    label_to_id = {label: idx for idx, label in enumerate(bio_labels)}

    # Tạo từ điển ánh xạ nhãn act_type thành chỉ số
    unique_act_types = [
        "None", "Attraction-Inform", "Attraction-NoOffer", "Attraction-Recommend", "Attraction-Select",
        "Booking-Book", "Booking-Inform", "Booking-NoBook", "Hospital-Inform", "Hotel-Inform", "Hotel-NoOffer", 
        "Hotel-Recommend", "Hotel-Select", "Police-Inform", "Restaurant-Inform", "Restaurant-NoOffer", 
        "Restaurant-Recommend", "Restaurant-Select", "Taxi-Inform", "Train-Inform", "Train-NoOffer", 
        "Train-OfferBook", "Train-OfferBooked", "Train-Select"
    ]
    act_type_to_id = {act_type: idx for idx, act_type in enumerate(unique_act_types)}

    # Lặp qua các dữ liệu train
    for item in data:
        # Câu thoại (utterance)
        utterance = item[2]  # Câu trong vị trí thứ 2

        # Các nhãn act_type và act_slot_name từ `dialogue_acts`
        dialogue_acts = item[3]  # Nhận phần dialogue_acts
        span_start = dialogue_acts["span_start"]
        span_end = dialogue_acts["span_end"]
        act_slot_name = dialogue_acts["act_slot_name"]
        act_type = dialogue_acts["act_type"]  # Nhãn act_type

        # Gán nhãn BIO cho các token trong câu
        tokens, bio_labels_for_tokens = create_bio_labels(
            utterance, {"span_start": span_start, "span_end": span_end}, act_slot_name
        )

        # Lưu các token và nhãn BIO
        X_token_classification.append(tokens)

        # Chuyển nhãn BIO thành chỉ số
        y_token_classification.append(
            [label_to_id[label] for label in bio_labels_for_tokens]
        )

        # Khởi tạo ma trận one-hot cho các hành động
        act_type_one_hot = np.zeros(len(unique_act_types))

        # Nếu có nhiều hành động, đặt giá trị 1 cho những hành động xuất hiện trong `act_type`
        for act in act_type:
            act_type_id = act_type_to_id[act]
            act_type_one_hot[act_type_id] = 1  # Gán 1 cho các vị trí của các hành động xuất hiện

        # Append vào y_act
        y_act.append(act_type_one_hot)

    return X_token_classification, y_token_classification, y_act


In [10]:
X_token_classification_train, y_token_classification_train, y_act_train = get_raw_data(data_train)
X_token_classification_val, y_token_classification_val, y_act_val = get_raw_data(data_validation)
X_token_classification_test, y_token_classification_test, y_act_test = get_raw_data(data_test)

# Get clean data

In [38]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, TimeDistributed, GlobalAveragePooling1D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

bio_labels = [
        "O",
        "B-address", "B-area", "B-arriveby", "B-bookday", "B-bookpeople", "B-bookstay", "B-booktime",
        "B-choice", "B-day", "B-department", "B-departure", "B-destination", "B-duration", "B-entrancefee", 
        "B-food", "B-leaveat", "B-name", "B-openhours", "B-phone", "B-postcode", "B-price", "B-pricerange", 
        "B-ref", "B-stars", "B-trainid", "B-type", "I-address", "I-area", "I-arriveby", "I-bookday", 
        "I-bookpeople", "I-bookstay", "I-booktime", "I-choice", "I-day", "I-department", "I-departure", 
        "I-destination", "I-duration", "I-entrancefee", "I-food", "I-leaveat", "I-name", "I-openhours", 
        "I-phone", "I-postcode", "I-price", "I-pricerange", "I-ref", "I-stars", "I-trainid", "I-type"
    ]
    
unique_act_types = [
        "None", "Attraction-Inform", "Attraction-NoOffer", "Attraction-Recommend", "Attraction-Select",
        "Booking-Book", "Booking-Inform", "Booking-NoBook", "Hospital-Inform", "Hotel-Inform", "Hotel-NoOffer", 
        "Hotel-Recommend", "Hotel-Select", "Police-Inform", "Restaurant-Inform", "Restaurant-NoOffer", 
        "Restaurant-Recommend", "Restaurant-Select", "Taxi-Inform", "Train-Inform", "Train-NoOffer", 
        "Train-OfferBook", "Train-OfferBooked", "Train-Select"
    ]

# Hyperparameters
EMBEDDING_DIM = 50
MAX_SEQUENCE_LENGTH = 50  # Giới hạn độ dài của câu
NUM_CLASSES_ACT = 24  # Số loại hành động
NUM_CLASSES_SLOT = len(bio_labels)  # Số nhãn BIO

# Load GloVe
def load_glove(glove_file):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Chuẩn bị Embedding Matrix
def create_embedding_matrix(word_index, embeddings_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [12]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(X_token_classification_train)
word_index = tokenizer.word_index

# Load GloVe
glove_file = '1/glove.6B.50d.txt'
embeddings_index = load_glove(glove_file)

# Create embedding matrix
embedding_matrix = create_embedding_matrix(word_index, embeddings_index, EMBEDDING_DIM)

# Convert sentences to sequences
X_train_seq = tokenizer.texts_to_sequences(X_token_classification_train)
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Prepare y_token_classification (convert to one-hot)
num_slot_labels = 53  # Số lượng nhãn token classification (sửa theo nhu cầu)
y_train_padded = pad_sequences(y_token_classification_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
y_train_padded_one_hot = np.array([to_categorical(y, num_classes=num_slot_labels) for y in y_train_padded])

# Prepare y_act (one-hot cho action classification)
y_act_one_hot = np.array(y_act_train)

# Chuẩn bị tập validation và test
X_val_seq = tokenizer.texts_to_sequences(X_token_classification_val)
X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
y_val_padded = pad_sequences(y_token_classification_val, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
y_val_padded_one_hot = np.array([to_categorical(y, num_classes=num_slot_labels) for y in y_val_padded])
y_act_val_one_hot = np.array(y_act_val)

X_test_seq = tokenizer.texts_to_sequences(X_token_classification_test)
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
y_test_padded = pad_sequences(y_token_classification_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
y_test_padded_one_hot = np.array([to_categorical(y, num_classes=num_slot_labels) for y in y_test_padded])
y_act_test_one_hot = np.array(y_act_test)


In [13]:
print(f"slot_output: y_train_padded_one_hot.shape={y_train_padded_one_hot.shape}, y_val_padded_one_hot.shape={y_val_padded_one_hot.shape}")
print(f"action_output: y_act_one_hot.shape={y_act_one_hot.shape}, y_act_val_one_hot.shape={y_act_val_one_hot.shape}")

# slot_output: y_train_padded_one_hot.shape=(113552, 50, 53), y_val_padded_one_hot.shape=(14748, 50, 53)
# action_output: y_act_one_hot.shape=(113552, 24), y_act_val_one_hot.shape=(14748, 24)


slot_output: y_train_padded_one_hot.shape=(113552, 50, 53), y_val_padded_one_hot.shape=(14748, 50, 53)
action_output: y_act_one_hot.shape=(113552, 24), y_act_val_one_hot.shape=(14748, 24)


In [22]:
# Model
input_seq = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedding = Embedding(
    input_dim=len(word_index) + 1,
    output_dim=EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False,
)(input_seq)

# BiLSTM
bilstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)

# Slot prediction
slot_output = TimeDistributed(
    Dense(NUM_CLASSES_SLOT, activation="softmax"), name="slot_output"
)(bilstm)

# Action prediction
sentence_representation = GlobalAveragePooling1D()(bilstm)
action_output = Dense(NUM_CLASSES_ACT, activation="softmax", name="action_output")(
    sentence_representation
)

# Model compilation
model = Model(inputs=input_seq, outputs=[slot_output, action_output])
model.compile(
    optimizer="adam",
    loss={
        "slot_output": "categorical_crossentropy",
        "action_output": "categorical_crossentropy",
    },
    metrics={"slot_output": ["accuracy"], "action_output": ["accuracy"]},
)

# Summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 50)    │    730,150 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 50, 128)   │     58,880 │ embedding_1[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ bidirectional_1[… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ slot_output         │ (None, 50, 53)    │      6,837 │ bidirectional_1[… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ action_output       │ (None, 24)        │      3,096 │ global_average_p… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 798,963 (3.05 MB)

 Trainable params: 68,813 (268.80 KB)

 Non-trainable params: 730,150 (2.79 MB)

In [ ]:
# output = model.predict(X_train_padded[:1])
# print("Slot output shape:", output[0].shape)  # Kết quả từ slot_output
# # print("Slot output:", output[0])  # Các nhãn của slot_output
# print("Action output shape:", output[1].shape)  # Kết quả từ action_output
# # print("Action output:", output[1])  # Các nhãn của action_output

# Slot output shape: (1, 50, 53)
# Action output shape: (1, 24)

In [23]:
history = model.fit(
    X_train_padded,  # Đầu vào cho mô hình (X_train_padded)
    [
        y_train_padded_one_hot,  # Đầu ra cho slot_output (shape: (None, 50, 53))
        y_act_one_hot,  # Đầu ra cho action_output (shape: (None, 24))
    ],
    validation_data=(
        X_val_padded,  # Đầu vào validation (X_val_padded)
        [
            y_val_padded_one_hot,  # Đầu ra validation cho slot_output (shape: (None, 50, 53))
            y_act_val_one_hot,  # Đầu ra validation cho action_output (shape: (None, 24))
        ],
    ),
    epochs=10,
    batch_size=64,
)


# Truy xuất metrics từ lịch sử huấn luyện
print("Available metrics:", history.history.keys())

Epoch 1/10
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 47s 25ms/step - action_output_accuracy: 0.1913 - action_output_loss: 1.3745 - loss: 1.8026 - slot_output_accuracy: 0.9521 - slot_output_loss: 0.4281 - val_action_output_accuracy: 0.2244 - val_action_output_loss: 1.2869 - val_loss: 1.5000 - val_slot_output_accuracy: 0.9574 - val_slot_output_loss: 0.2137
Epoch 2/10
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 50s 28ms/step - action_output_accuracy: 0.2359 - action_output_loss: 1.2608 - loss: 1.4680 - slot_output_accuracy: 0.9595 - slot_output_loss: 0.2073 - val_action_output_accuracy: 0.2670 - val_action_output_loss: 1.2449 - val_loss: 1.4442 - val_slot_output_accuracy: 0.9597 - val_slot_output_loss: 0.1997
Epoch 3/10
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 57s 32ms/step - action_output_accuracy: 0.2382 - action_output_loss: 1.2503 - loss: 1.4424 - slot_output_accuracy: 0.9619 - slot_output_loss: 0.1921 - val_action_output_accuracy: 0.2280 - val_action_output_loss: 1.2755 - val_loss: 1.4664 - val_slot_output_accuracy: 

In [ ]:
# Save the model
model.save("s1_model.h5")

In [ ]:
from sklearn.metrics import classification_report

# Load the model
model = tf.keras.models.load_model("s1_model.h5")

# Đánh giá trên tập test
print("Evaluating on test set...")
results = model.evaluate(X_test_padded, [y_test_padded_one_hot, y_act_test_one_hot])

# Dự đoán trên tập test
y_pred_slot, y_pred_action = model.predict(X_test_padded)

In [53]:
# Chuyển y_pred_slot và y_pred_action về nhãn
y_pred_slot_labels = np.argmax(y_pred_slot, axis=-1)  # Slot prediction
y_pred_action_labels = np.argmax(y_pred_action, axis=-1)  # Action prediction
y_true_slot_labels = np.argmax(y_test_padded_one_hot, axis=-1)
y_true_action_labels = np.argmax(y_act_test_one_hot, axis=-1)

y_pred_slot_flatten = y_pred_slot_labels.reshape(-1)
y_true_slot_flatten = y_true_slot_labels.reshape(-1)

print(y_pred_slot.shape, y_pred_action.shape)
print(y_pred_slot_labels.shape, y_pred_action_labels.shape)
print(y_true_slot_labels.shape, y_true_action_labels.shape)
print(y_pred_slot_flatten.shape, y_true_slot_flatten.shape)

(14744, 50, 53) (14744, 24)
(14744, 50) (14744,)
(14744, 50) (14744,)
(737200,) (737200,)


In [ ]:
print("Slot Classification Metrics:")
print(classification_report(y_true_slot_flatten, y_pred_slot_flatten, zero_division=0))

# Slot Classification Metrics:
#               precision    recall  f1-score   support

#            0       0.96      1.00      0.98    703893
#            1       0.38      0.04      0.06       566
#            2       0.52      0.14      0.22      1808
#            3       0.40      0.10      0.15       821
#            4       0.34      0.05      0.08       644
#            5       0.35      0.65      0.46       941
#            6       0.78      0.12      0.21       355
#            7       0.53      0.18      0.26       419
#            8       0.55      0.36      0.43      1276
#            9       0.14      0.03      0.05       640
#           11       0.32      0.02      0.04       777
#           12       0.00      0.00      0.00       835
#           13       0.00      0.00      0.00       122
#           14       0.00      0.00      0.00       283
#           15       0.46      0.14      0.22       861
#           16       0.30      0.23      0.26       898
#           17       0.60      0.06      0.12      2165
#           19       0.00      0.00      0.00       618
#           20       0.00      0.00      0.00       380
#           21       0.31      0.02      0.03       332
#           22       0.23      0.01      0.02      1146
#           23       0.00      0.00      0.00       811
#           24       0.20      0.01      0.01       434
#           25       0.61      0.09      0.16       445
#           26       0.54      0.04      0.07      1561
#           27       0.25      0.01      0.03      1818
#           28       0.00      0.00      0.00       793
#           29       0.33      0.00      0.00      1630
#           30       0.00      0.00      0.00        24
#           31       0.00      0.00      0.00        67
#           32       0.00      0.00      0.00         3
#           33       0.79      0.15      0.25       824
#           34       0.00      0.00      0.00       199
#           35       0.00      0.00      0.00        16
#           37       0.00      0.00      0.00       447
#           38       0.00      0.00      0.00       467
#           39       0.82      0.07      0.13       131
#           40       0.00      0.00      0.00       379
#           41       0.50      0.02      0.03       133
#           42       0.24      0.25      0.24      1849
#           43       0.44      0.09      0.15      3887
#           45       0.00      0.00      0.00        10
#           46       0.00      0.00      0.00         1
#           47       0.51      0.07      0.13       952
#           48       0.00      0.00      0.00       191
#           49       0.00      0.00      0.00         1
#           50       0.00      0.00      0.00         6
#           51       0.00      0.00      0.00         4
#           52       0.00      0.00      0.00       337

#     accuracy                           0.96    737200
#    macro avg       0.25      0.08      0.10    737200
# weighted avg       0.94      0.96      0.94    737200


Slot Classification Metrics:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    703893
           1       0.38      0.04      0.06       566
           2       0.52      0.14      0.22      1808
           3       0.40      0.10      0.15       821
           4       0.34      0.05      0.08       644
           5       0.35      0.65      0.46       941
           6       0.78      0.12      0.21       355
           7       0.53      0.18      0.26       419
           8       0.55      0.36      0.43      1276
           9       0.14      0.03      0.05       640
          11       0.32      0.02      0.04       777
          12       0.00      0.00      0.00       835
          13       0.00      0.00      0.00       122
          14       0.00      0.00      0.00       283
          15       0.46      0.14      0.22       861
          16       0.30      0.23      0.26       898
          17       0.60      0.06      0.12      216

In [ ]:
print("Action Classification Metrics:")
print(classification_report(y_true_action_labels, y_pred_action_labels, zero_division=0))

# Action Classification Metrics:
#               precision    recall  f1-score   support

#            0       0.00      0.00      0.00      5491
#            1       0.48      0.03      0.05      1431
#            2       0.00      0.00      0.00        42
#            3       0.00      0.00      0.00        94
#            4       0.00      0.00      0.00         7
#            5       0.00      0.00      0.00       528
#            6       0.00      0.00      0.00        38
#            7       0.00      0.00      0.00        28
#            9       0.25      0.13      0.17      1867
#           10       0.00      0.00      0.00        41
#           11       0.00      0.00      0.00        99
#           12       0.00      0.00      0.00        12
#           13       0.00      0.00      0.00         2
#           14       0.12      0.68      0.20      1885
#           15       0.00      0.00      0.00        70
#           16       0.00      0.00      0.00        98
#           17       0.00      0.00      0.00        12
#           18       0.00      0.00      0.00       452
#           19       0.50      0.66      0.57      2236
#           20       0.00      0.00      0.00         5
#           21       0.00      0.00      0.00        26
#           22       0.00      0.00      0.00       276
#           23       0.00      0.00      0.00         4

#     accuracy                           0.21     14744
#    macro avg       0.06      0.07      0.04     14744
# weighted avg       0.17      0.21      0.14     14744

Action Classification Metrics:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5491
           1       0.48      0.03      0.05      1431
           2       0.00      0.00      0.00        42
           3       0.00      0.00      0.00        94
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00       528
           6       0.00      0.00      0.00        38
           7       0.00      0.00      0.00        28
           9       0.25      0.13      0.17      1867
          10       0.00      0.00      0.00        41
          11       0.00      0.00      0.00        99
          12       0.00      0.00      0.00        12
          13       0.00      0.00      0.00         2
          14       0.12      0.68      0.20      1885
          15       0.00      0.00      0.00        70
          16       0.00      0.00      0.00        98
          17       0.00      0.00      0.00       

In [ ]:
# Tạo câu mẫu để test (có thể thay đổi câu mẫu tùy ý)
sample_sentence = ["I want to use train to Hanoi"]

# Chuyển câu mẫu thành chuỗi số
sample_sequence = tokenizer.texts_to_sequences(sample_sentence)

# Padding chuỗi số để đạt độ dài như trong mô hình
sample_padded = pad_sequences(sample_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Pass qua mô hình để dự đoán
slot_prediction, action_prediction = model.predict(sample_padded)

slot_prediction = np.argmax(slot_prediction, axis=-1)
action_prediction = np.argmax(action_prediction, axis=-1)

# In ra kết quả dự đoán
print("Slot Prediction:", slot_prediction, [bio_labels[slot] for slot in slot_prediction[0]])
print("Slot Prediction Shape:", slot_prediction.shape)

print("Action Prediction:", action_prediction, unique_act_types[action_prediction[0]])
print("Action Prediction Shape:", action_prediction.shape)

# Slot Prediction: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
#   0 0 0 0 0 0 0 0 0 0 0 0 0 0]] ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# Slot Prediction Shape: (1, 50)
# Action Prediction: [14] Restaurant-Inform
# Action Prediction Shape: (1,)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Slot Prediction: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]] ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Slot Prediction Shape: (1, 50)
Action Prediction: [14] Restaurant-Inform
Action Prediction Shape: (1,)
